In [1]:
import torch
import pickle
import torch.nn as nn
import torch.nn.functional as F
import os
import sys
import click
import dnnlib
import legacy
import numpy as np
import time

device = "cuda:1"
size = 1024
truncation_psi = 0.5
n_iters = 50001
batch_size = 1
random_start_seed = 0
iter_start_seed = 0
# d = torch.Tensor(np.random.RandomState(random_start_seed).normal(0, 0.2, [1, 512])).to(device)
d = torch.zeros([1, 512]).to(device)

target_feature_index = 4
epsilon_const = False
epsilon_value = 0.5
reg_loss_flag = 'mse' # bce or mse
lambda_regressor = 10.0
lambda_content = 0.00
lambda_gan = 0.00
learning_rate = 0.001

noise_mode = 'const'


In [2]:
network_pkl = './ffhq.pkl'

with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

with dnnlib.util.open_url(network_pkl) as f:
    D = legacy.load_network_pkl(f)['D'].to(device)
    
label = torch.zeros([1, G.c_dim], device=device)

In [3]:
regressor = torch.jit.load('resnet50.pth').eval().to(device)

In [11]:
import time
time1 = time.time()
z_list = []
w_list = []
pred_list = []
with torch.no_grad():
    for random_seed in range(500001):
        z = torch.from_numpy(np.random.RandomState(random_seed).randn(batch_size, 512)).to(device)
        w = G.mapping(z,label, truncation_psi=truncation_psi)  
        img = G.synthesis(w, noise_mode=noise_mode)
        z_list.append(z[0,:].cpu().detach().numpy())
        w_list.append(w[0,0,:].cpu().detach().numpy())
        img = F.interpolate(img, size=256)
        pred = regressor(img)
        pred_list.append(pred[0,:].cpu().detach().numpy())
        if random_seed % 1000 == 0:
            np.save('./interfacegan_data/z/z_iter_' + str(random_seed) + '.npy', np.array(z_list))
            np.save('./interfacegan_data/w/w_iter_' + str(random_seed) + '.npy', np.array(w_list))
            np.save('./interfacegan_data/pred/pred_iter_' + str(random_seed) + '.npy', np.array(pred_list))
            z_list = []
            w_list = []
            pred_list = []
            print(random_seed)
            
time2 = time.time()
print(time2 - time1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


# Combine all l_vec_files into one

In [41]:
from os import listdir
from os.path import isfile, join
from natsort import natsorted


path = './interfacegan_data/pred/'

In [42]:
def stack_all(path):
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    l_vec_list = natsorted(onlyfiles)
    def stack_l_vecs(l_vec_list, start_idx, end_idx):
        a = np.load(path + l_vec_list[start_idx])
        for l_vec in l_vec_list[start_idx:end_idx]:
            b = np.load(path + l_vec)
            a = np.vstack([a,b])
        return a

    l_vec_1 = stack_l_vecs(l_vec_list, 0, 100)
    l_vec_2 = stack_l_vecs(l_vec_list, 100, 200)
    l_vec_3 = stack_l_vecs(l_vec_list, 200, 300)
    l_vec_4 = stack_l_vecs(l_vec_list, 300, 400)
    l_vec_5 = stack_l_vecs(l_vec_list, 400, -1)
    all_l_vecs = np.vstack([l_vec_1, l_vec_2, l_vec_3, l_vec_4, l_vec_5])
    return all_l_vecs

all_z = stack_all('./interfacegan_data/z/')
all_w = stack_all('./interfacegan_data/w/')
all_pred = stack_all('./interfacegan_data/pred/')

In [56]:
np.save('./interfacegan_data/all_z.npy', all_z[0:500000])
np.save('./interfacegan_data/all_w.npy', all_w[0:500000])
np.save('./interfacegan_data/all_pred.npy', all_pred[0:500000])

In [57]:
np.save('./interfacegan_data/smiling_31_pred.npy', all_pred[0:500000, 31:32])
np.save('./interfacegan_data/hair_col_9_pred.npy', all_pred[0:500000, 9:10])